In [1]:
# Install a pip comtradeapicall package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install --upgrade comtradeapicall

/Users/oscardybenkoguajardomorales/.zshenv:3: no such file or directory: 11.jdk/Contents/Home/
/Users/oscardybenkoguajardomorales/.zshenv:6: no such file or directory: 11.jdk/Contents/Home/
/Users/oscardybenkoguajardomorales/.zshenv:export:8: not an identifier: 11.jdk/Contents/Home/


In [3]:
import pandas as pd
import os 
import requests
from dotenv import load_dotenv


In [5]:


# Loading API
load_dotenv()
subscription_key = os.getenv("un_key")

# Parameers
typeCode='C' #Commodities data
freqCode='A' # Annual frequency
clCode='HS' # Harmonized System classification
countries_codes = ['84', '188', '222', '320', '340', '484', '558', '591', '32', '68', '76', '152', '170', '218', '600', '604', '858', '862']
reporter = ",".join(countries_codes)
flowCode="X" # Only Import or export at once


# Preparing request
request_url = f"https://comtradeapi.un.org/data/v1/get/{typeCode}/{freqCode}/{clCode}"
headers = {'Ocp-Apim-Subscription-Key': subscription_key}
params = {
    "reporterCode": reporter,
    "period": 2023,
    "partnerCode": reporter + ",0",
    "partner2Code": "0",
    "cmdCode": "TOTAL",
    "flowCode": flowCode,
    "customsCode": "C00",
    "motCode" : 0,
    }

countries_dict = {
    84: "Belize",
    188: "Costa Rica",
    222: "El Salvador", 
    320: "Guatemala",
    340: "Honduras",
    484: "Mexico",
    558: "Nicaragua",
    591: "Panama",
    32: "Argentina",
    68: "Bolivia",
    76: "Brazil",
    152: "Chile",
    170: "Colombia",
    218: "Ecuador",
    600: "Paraguay",
    604: "Peru",
    858: "Uruguay",
    862: "Venezuela"
}


# Doing request
response = requests.get(url=request_url, params=params, headers=headers)
if response.status_code == 200:
    print("Retrieving export data... Succesful!\n")
else:
    print("Error retrieving data: {}".format(response.json()))

# Storing in DF
data = pd.DataFrame(response.json()["data"])

# Now doing it for imports
params["flowCode"] = "M"
response = requests.get(url=request_url, params=params, headers=headers)
if response.status_code == 200:
    print("Retrieving import data... Succesful!\n")
data = pd.concat([data, pd.DataFrame(response.json()["data"])])


# Summarizing data
required_cols = ["reporterCode", "flowCode", "partnerCode", "primaryValue"]
summary_data = data.loc[:, required_cols]
# Classifying partner countries as Latam or not Latam
def find_country(valor):
    return countries_dict.get(valor, "Not LatinAmerica")

summary_data["rep_country"] = summary_data["reporterCode"].apply(find_country)
summary_data["part_country"] = summary_data["partnerCode"].apply(find_country)
summary_data["NotLATAM"] = summary_data["part_country"] == "Not LatinAmerica"

Retrieving export data... Succesful!

Retrieving import data... Succesful!



Como hay datos agregados (comercio total con el codigo 0 o comercio con centro y nortamerica con el codigo 637, rest of america 636, caribe 129)
Tambien se doble cuenta si no tomo en consideracion los medios de transporte
Ademas veo que no funciona bien la funcion de agregar porque al momento de sumar los c[odigos CMD parece que tambien hay un dobleo conteo.

Mi solucion es Pedir datos con el CMD code de Total porque sino son demasiadas lineas y no me da todos los datos. As[i que ya es una forma de agregar sin explicitamente pedirle a la API que calcule el agregado por CMD.

Despues calculo el comercio total con los paises de latam y luego comparo eso con el comercio total con el codigo 0 de partnerCountry. Asi saco implicitament el comercio con paises fuera de latam.



In [495]:
summary_data.columns

Index(['reporterCode', 'flowCode', 'partnerCode', 'primaryValue',
       'rep_country', 'part_country', 'NotLATAM'],
      dtype='object')

In [500]:
# Sumando import y exports 
summary_data = summary_data[['reporterCode', 'partnerCode',
       'rep_country', 'part_country', 'NotLATAM', 'primaryValue']].groupby(['reporterCode', 'partnerCode',
       'rep_country', 'part_country', 'NotLATAM']).sum().reset_index()

In [519]:
# Datos de comercio intra-Latinoamerica
summary_data.loc[summary_data["partnerCode"] != 0, ["rep_country", "primaryValue"]].groupby("rep_country").sum().sort_values(by="rep_country").to_clipboard()
summary_data.loc[summary_data["partnerCode"] != 0, ["rep_country", "primaryValue"]].groupby("rep_country").sum().sort_values(by="rep_country")

,primaryValue
rep_country,
Argentina,7.910283e+10
Belize,3.821439e+08
Bolivia,1.037518e+10
Brazil,8.863655e+10
Chile,3.532050e+10
Colombia,3.861421e+10
Costa Rica,8.838386e+09
Ecuador,1.578082e+10
El Salvador,9.176206e+09


In [520]:
# Datos de comercio total (latam + no latam)
summary_data.loc[summary_data["partnerCode"] == 0, ["rep_country", "primaryValue"]].sort_values(by="rep_country").to_clipboard()
summary_data.loc[summary_data["partnerCode"] == 0, ["rep_country", "primaryValue"]]

,rep_country,primaryValue
0,Argentina,2.142157e+11
19,Bolivia,2.240798e+10
38,Brazil,5.924062e+11
57,Belize,1.590856e+09
74,Chile,1.804407e+11
93,Colombia,1.753302e+11
111,Costa Rica,4.061801e+10
129,Ecuador,6.202210e+10
147,El Salvador,2.214642e+10
166,Guatemala,4.453901e+10


In [11]:
summary_data.head(1)

,reporterCode,flowCode,partnerCode,primaryValue,rep_country,part_country,NotLATAM
0,32,X,0,6.678852e+10,Argentina,Not LatinAmerica,True


In [12]:
summary_data.loc[summary_data["rep_country"] == "Mexico", ["partnerCode", "primaryValue"]].groupby("partnerCode").sum().sort_values(by="primaryValue", ascending=False)

,primaryValue
partnerCode,
0,1.191475e+12
76,1.793114e+10
170,5.008206e+09
320,3.660660e+09
152,3.358849e+09
604,2.792371e+09
188,2.547359e+09
340,2.244875e+09
558,2.212427e+09
